In [ ]:
# Clone the repository and install the necessary packages
!git clone https://github.com/JakubMartinka/karlsruhe2024.git
!pip3 install mlatom
!pip3 install mkl
!pip3 install mkl-service
!pip3 install py3Dmol
!pip3 install geometric
!pip3 install pyscf==2.4.0

Cloning into 'karlsruhe2024'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (90/90), done.


In [ ]:
import mlatom as ml

In [ ]:
# get initial structure
h2 = ml.molecule.from_xyz_string('''2

H 0 0 0
H 0 0 0.7
''')

In [ ]:
# define the reference method
dft = ml.models.methods(method='B3LYP/6-31G*')

In [ ]:
# optimize geometry and calculate frequency
#optmol = ml.optimize_geometry(
#    model=dft,
#    initial_molecule=h2
#    ).optimized_molecule
#freq = ml.freq(model=dft,vmolecule=optmol)

optmol = ml.molecule.load('karlsruhe2024/materials/h2_freq.json')

In [ ]:
# settings for sampling points during AL loop
sampler_kwargs = {
    'initcond_sampler':'wigner',
    'initcond_sampler_kwargs':{
        'molecule':optmol,
        'number_of_initial_conditions':100,
        'initial_temperature':300},
    'maximum_propagation_time':100.0,
    'time_step':0.5,
    'uq_threshold':0.000001,
}

In [ ]:
# start active learning
ml.al(molecule=optmol,
      ml_model='KREG',
      sampler = 'md',
      sampler_kwargs=sampler_kwargs,
      min_new_points=5,
      reference_method=dft)

In [ ]:
# check the final labeled database
db = ml.molecular_database.load(filename='labeled_db.json', format='json')
len(db)

In [ ]:
for imol in range(len(db)):
    print(db[imol].internuclear_distance(0, 1), db[imol].energy)

In [ ]:
db.view()